In [171]:
from bs4 import BeautifulSoup
import re
import os
import json
import pandas as pd

In [164]:
DATA_PATH = "raw_data/"

In [165]:
articles = []
for i,file_name in enumerate(os.listdir(DATA_PATH)):
    print(f"======={i}======")
    with open(DATA_PATH+file_name, "r", encoding='utf-8') as f:
         html_doc= f.read()
    soup = BeautifulSoup(html_doc, 'html.parser')
    # Get table that contains the information we are interested in
    table = soup.find("table", {"id": "filterrific_results"})
    # get tags that contain the information
    rows = [content  for content in table.contents if content.name]
    # split the tags per teacher
    teachers = []
    for i in range(0,len(rows),3):
        teacher = [rows[i],rows[i+2]]
        teachers.append(teacher)
    for teacher in teachers:
         # Pulling name and grade of each teacher:  'Enseignant NAME (GRADE)'
         text = teacher[0].th.text
         text = re.split(pattern='Enseignant', string=text)[1][2:]
         text = re.split(pattern='\(', string=text)
         name = text[0].strip()
         grade = text[1][:-1]
         # splitting the data per article
         articles_teacher = teacher[1].find_all('tr')
         print(f'{name}:{len(articles_teacher)}')
         # populating  articles information into a dictionary
         for article in articles_teacher:
              infos = article.find_all('td')
              date, time, _ = re.split(pattern=' ', string=infos[9].text)
              article_dict = {
                  'Id': infos[0].text,
                  'Title': infos[1].text.strip(),
                  'Author': name,
                  'Grade': grade,
                 'Taken': bool(infos[2].text.strip()),
                 'Priority 1': infos[3].text.strip(),
                  'Priority 2': infos[4].text.strip(),
                  'Priority 3': infos[5].text.strip(),
                   'Priority 4': infos[6].text.strip(),
                    'Priority 5': infos[7].text.strip(),
                   'Date': date,
                          'Time': time,
              }
              articles.append(article_dict)



=======0======
KHABABA  Abdallah:4
ALIOUAT  Makhlouf:2
KAMEL  Nadjet:2
MOUSSAOUI  Abdelouahab:11
REFOUFI  Allaoua:1
=======1======
REFOUFI  Allaoua:1
SAIDI  Mohamed:2
ALIOUAT née ZOUAOUI  Zibouda:2
KHENTOUT  Chabane:1
DOUIDI  Lamri:3
BENAOUDA  Abdelhafid:3
KHENTOUT née HARBOUCHE  Khadidja:2
ZAIMEN née MEDIANI  Chahrazed:3
MECHTA  Djamila:3
=======2======
MECHTA  Djamila:1
LAOUADI  Mohamed Amin:2
BEGHRICHE  Abdesselem:2
AISSAOUA  Habib:2
BENAIDJA  Amira:2
MEKROUD  Noureddine:3
KHARCHI  Samia:2
HAMMOUCHE  Yassine:2
FENANIR  Samir:3
LAKHFIF  Abdelaziz:1
=======3======
LAKHFIF  Abdelaziz:2
TOUMI  Lyazid:4
LALAMA  Zahia:1
DJAGHLOUL  Haroun:2
BANABIREZ née ZERGUINE  Nadia:2
DRIF  Ahlem:2
AMRANI  Lakhdar:2
BOUAMARI  Abdelkader:2
ZAZOUA née DJEMAME  Safia:2
DOUAR  Amel:1
=======4======
DOUAR  Amel:1
GOUDJIL  Lakhdar:2
KHALFALLAH née HADI  Fairouz:3
GUELLATI  Nabil:1
SEMCHEDINE  Moussa:2
BRAHIM-SALEM  Fatiha:2
KARA  Abdelaziz:2
MANSOURI  Houceme:2
NASRI  Khaled:2
TEBBANI  Fatiha:2
KANOUNI  Lakh

In [166]:
assert len(articles) == 147

In [169]:
# Saving
with open('dataset/result.json', 'w') as fp:
    json.dump(articles, fp)

In [172]:

df = pd.read_json (r'dataset/result.json')
df.to_csv (r'dataset/sujets_master_2022.csv', index = None)

In [173]:
df = pd.read_csv('dataset/sujets_master_2022.csv')

In [174]:
df

,Id,Title,Author,Grade,Taken,Priority 1,Priority 2,Priority 3,Priority 4,Priority 5,Date,Time
0,1499,IoT for remote monitoring in Algeria,KHABABA Abdallah,Professeur,True,F3I,IDTW,IFIA,GL,RSD,2022-01-04,09:49:23
1,1501,"Development, Usability, and Efficiancy of a Se...",KHABABA Abdallah,Professeur,True,GL,IFIA,F3I,IDTW,RSD,2022-01-02,19:21:14
2,1502,Conception et Réalisation d’un système de réal...,KHABABA Abdallah,Professeur,True,F3I,RSD,IDTW,GL,IFIA,2022-01-16,12:13:05
3,1504,An IoT Based Mobile Augmented Reality Applicat...,KHABABA Abdallah,Professeur,True,F3I,IDTW,GL,IFIA,RSD,2022-01-04,09:50:25
4,1363,Architecture d’un Système Nano Macro de Réseau...,ALIOUAT Makhlouf,Professeur,True,RSD,IDTW,IFIA,GL,F3I,2022-01-02,19:41:53
...,...,...,...,...,...,...,...,...,...,...,...,...
142,1376,Internet of Things Energy-Efficient routing pr...,Zier Abdelhak,Maitre assistant classe B,True,RSD,IFIA,IDTW,F3I,GL,2022-01-03,20:20:51
143,1483,Authentification sécurisée dans l’internet des...,Harbi Yasmine,Maitre assistant classe B,True,RSD,IFIA,IDTW,F3I,GL,2022-01-03,18:37:04
144,1413,Une approche de sélection des caractéristiques...,BERRHAIL Fouaz,Maitre de conferences classe B) --> Hors dépar...,True,IFIA,IDTW,F3I,GL,NaN,2022-01-25,12:49:59
145,1414,Réalisation d’un système de détection des mala...,BERRHAIL Fouaz,Maitre de conferences classe B) --> Hors dépar...,True,IDTW,IFIA,F3I,GL,RSD,2022-01-04,20:17:34


## Question I want to know their answer:

1. numer of subjects
2. percentage of subjects affected/unaffected
3. number of teachers who proposed a subject
4. rank teacher per number of subjects
5. rank subjects (close ones) per teacher
6. rank speciality's per number of subjects/teachers
7. sum of priority's for every speciality